In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import panel as pn
import pickle
import ExplorePickle as EP
pn.extension('tabulator')


from bokeh.models.widgets.tables import DateFormatter
bokeh_formatters = {
    'LastSale': DateFormatter(format= "%d %b %Y"),
}
#np.random.seed(7)


pickledir = 'pickles/{0}'
df = pd.read_pickle(pickledir.format('2023-12-22.pkl'))
grandTotal = pd.read_pickle(pickledir.format('GrandTotals23-12-20.pkl'))
cityTotal = pd.read_pickle(pickledir.format('CityTotals23-12-20.pkl'))
OwnerNamePresetDir = pickledir.format('OwnerNamePresets23-12-20.pkl')
geodictionarypath = pickledir.format("geodict23-12-22.pkl")

showcolumns = ['Location','City','Description','Style','Bedrooms','Owner','Owner2','Owner3',
               'OwnerAdd','OwnerAdd2','OwnerAdd3','Assessment','LastSale']
tablewidths = {'Location':150,'Description':100,'Style':100,'Bedrooms':70,'Owner':150,'Owner2':100,'Owner3':100,
               'OwnerAdd':150,'OwnerAdd2':100,'OwnerAdd3':100,'Assessment':100,'LastSale':100}
df = df[showcolumns]

In [2]:
#MAIN TAB
#
#
#
def searchowners(df,searchstring,searchallowners = True):
    #Search by Owners
    #returns dataframe
    ownercolumns = ['Owner','Owner2','Owner3']
    searchresult = []
    for column in ownercolumns:
        searchresult.append(df.loc[df[column].str.contains('(?i){0}'.format(searchstring))])
    searchresult = pd.concat(searchresult)
    return searchresult
def searchcities(df,searchlist):
    searchresult = []
    for searchterm in searchlist:
        if searchterm == 'all':
            return df
        searchresult.append(df.loc[df.City == searchterm])
    return pd.concat(searchresult)
def searchproplocation(df,searchstring):
    if len(searchstring)>2:
        return df.loc[df.Location.str.contains(searchstring,case=False)]
    else:
        return df
def searchowneradd(df,searchstring):
    if len(searchstring)>2:
        return df.loc[df.OwnerAdd.str.contains(searchstring,case=False)]
    else:
        return df
def searchownertype(df, searchstring):
    if searchstring == 'all':
        return 
    else:
        FindType = EP.OwnerTypes[searchstring]
        searchresult = FindType(df)
        return searchresult
        
def updateDF(event):
    updatedDF = df
    ownercolumns = ['Owner','Owner2','Owner3']
    if presetselect.value != '':
        #preset section
        with open(OwnerNamePresetDir, 'rb') as f:
            OwnerIndices = pickle.load(f)
        OwnerIndices = OwnerIndices[presetselect.value] #Should be a list
        updatedDF = updatedDF.set_index(ownercolumns).loc[OwnerIndices]
        #updatedDF = searchowners(updatedDF,searchterm)
        df_widget.value = updatedDF.reset_index(drop = False)
        return
    cityvalues = citycheckboxes1.value + citycheckboxes2.value #which cities to search
    if cityvalues[0] == 'all':
        citycheckboxes1.value = ['all']
        citycheckboxes2.value = []
    else:
        updatedDF = searchcities(updatedDF,cityvalues)
    if ownertypeselect.value != 'all':
        updatedDF = searchownertype(updatedDF,ownertypeselect.value)
    updatedDF = searchowners(updatedDF,ownersearch.value)
    updatedDF = searchproplocation(updatedDF,locationsearch.value)
    updatedDF = searchowneradd(updatedDF,owneraddresssearch.value)
    df_widget.value = updatedDF
    return
    
def resetDF(event):
    df_widget.value = df
    citycheckboxes1.value = ['all']
    citycheckboxes2.value = []
    owneraddresssearch.value = ''
    ownersearch.value = ''
    locationsearch.value = ''
    ownertype.value = ['all']

df_widget = pn.widgets.Tabulator(df,frozen_columns=['Location'],show_index=False,width = 1000,
                                 formatters=bokeh_formatters,widths = tablewidths,page_size = 20)


updatebutton = pn.widgets.Button(name='Update')
resetbutton = pn.widgets.Button(name='Reset')
downloadbutton = pn.widgets.Button(name='Download')


ownersearch = pn.widgets.TextInput(value = '',name = 'Search by Owner')

citycheckboxes1 = pn.widgets.CheckBoxGroup(
    value=['all'], options=['all',*[key for key in EP.dbtype.keys()][:17]])
citycheckboxes2 = pn.widgets.CheckBoxGroup(
    value=['all'], options=[key for key in EP.dbtype.keys()][17:])
citycheckboxes = [citycheckboxes1,citycheckboxes2]
cityselect = pn.Accordion(pn.Row(*citycheckboxes, name = 'filter by city'),name = 'cities')

ownertypeselect = pn.widgets.Select(name='Filter by owner type',
                    value='all', options=['all','State','NonState'])
presetselect = pn.widgets.Select(name = 'Presets',value = '',options = ['',*[key for key in EP.replacedict]])

accordionrow = pn.Row(cityselect,ownertypeselect,presetselect)

locationsearch = pn.widgets.TextInput(value = '',name = 'Search by Property Location')
owneraddresssearch = pn.widgets.TextInput(value = '',name = 'Search by Owner Address')
debug = pn.widgets.TextInput(value = '',name = 'debug')

#dfparams = (citycheckboxes1.value,citycheckboxes2.value,
#           ownersearch,locationsearch,owneraddresssearch)
    
updatebutton.on_click(updateDF)
resetbutton.on_click(resetDF)

propertytab = pn.Column(accordionrow,
              pn.Row(ownersearch,locationsearch,owneraddresssearch),
              pn.Row(updatebutton, resetbutton, downloadbutton),
              df_widget,
             debug, name = 'Properties')
#
#
#Summary Tab
#
#
def MakeSummary(inputdf):
    """Takes owner values of the main dataframe,
    and searches in the summary dataframe for those owners"""
    ownerlist = inputdf['Owner'].unique()
    if summarizecheck.value == True:
        if grandtotalcheck.value == True:
            return  grandTotal.loc[ownerlist].groupby('City').sum()
        else:
            return grandTotal.loc[ownerlist]
        return grandTotal.loc[ownerlist]
    else:
        return cityTotal.loc[ownerlist]
def sumbuttonclick(event):
    sum_df.value = MakeSummary(df_widget.value)
    
    return
    
sum_df = pn.widgets.Tabulator(grandTotal,width = 1000,
                                 formatters=bokeh_formatters,page_size = 20)

summarizebutton = pn.widgets.Button(name = "Summarize")
summarizecheck = pn.widgets.Checkbox(value = True,name = 'Totals')
grandtotalcheck = pn.widgets.Checkbox(value = False,name = 'GrandTotal')

summarizebutton.on_click(sumbuttonclick)
summarytab = pn.Column(summarizebutton,summarizecheck,grandtotalcheck,sum_df,name = 'Summaries')

#
#
#
#
##MAP TAB
###
import folium
import geopandas
from shapely.geometry import Point

def loadpoints(event):
    newdf = df_widget.value
    if len(df)>300:
        newdf = newdf[:300]
    #pointindex = df_widget.index.to_list
    with open(geodictionarypath, 'rb') as f:
        geodictionary = pickle.load(f)
    searchterms = getsearchterms(newdf)
    coords = searchterms.apply(getcoords, geodict = geodictionary)
    newdf['Coords'] = coords.apply(Point)
    newdf = geopandas.GeoDataFrame(newdf,geometry = "Coords",crs = "EPSG:4326")
    #folium.Map(newdf).add_to(m)
    folium.GeoJson(newdf,na='drop',
                   marker=folium.CircleMarker(
                       radius=4, fill_color="orange", fill_opacity=0.4, color="black", weight=1)
                  ).add_to(m)
    folium_pane.object = m
    #dfgeoseries = geopandas.GeoSeries(coords)
    #df_widget.value =  newdf
    #Then load the layer into folium

def getsearchterms(inputdf):
    """returns searchaddress, which can be used with geodict to
    get coords"""
    searchterms = inputdf['Location']+', '+inputdf['City'].replace(EP.citynamedict) +', RI, '
    return searchterms
    
def getcoords(searchaddy, geodict):
    """Takes a search term and returns coordinate points"""
    try:
        return geodict[searchaddy]
    except:
        print(searchaddy)
        return (np.nan,np.nan)    
        

loadptsbutton = pn.widgets.Button(name = 'Load pts')
loadptsbutton.on_click(loadpoints)
m = folium.Map(location=[41.82, -71.4], zoom_start=11)
folium_pane = pn.pane.plot.Folium(m, height=400)
maptab = pn.Column(loadptsbutton,folium_pane, name = 'Map')

#
#
#
pn.Tabs(propertytab,summarytab,maptab)


Tabs
    [0] Column(name='Properties')
        [0] Row
            [0] Accordion(name='cities')
                [0] Row(name='filter by city')
                    [0] CheckBoxGroup(options=['all', 'Barrington', ...], value=['all'])
                    [1] CheckBoxGroup(options=['Narragansett', ...])
            [1] Select(name='Filter by owner type', options=['all', 'State', ...], value='all')
            [2] Select(name='Presets', options=['', 'stonelink and greenf...])
        [1] Row
            [0] TextInput(name='Search by Owner')
            [1] TextInput(name='Search by Property L...)
            [2] TextInput(name='Search by Owner Address')
        [2] Row
            [0] Button(name='Update')
            [1] Button(name='Reset')
            [2] Button(name='Download')
        [3] Tabulator(formatters={'LastSale': DateFormatter...}, frozen_columns=['Location'], pagination='remote', show_index=False, value=              ..., width=1000, widths={'Location': 150, ...})
        [4] TextInput(name='debug')
    [1] Column(name='Summaries')
        [0] Button(name='Summarize')
        [1] Checkbox(name='Totals', value=True)
        [2] Checkbox(name='GrandTotal')
        [3] Tabulator(formatters={'LastSale': DateFormatter...}, pagination='remote', value=              ..., width=1000)
    [2] Column(name='Map')
        [0] Button(name='Load pts')
        [1] Folium(Map, height=400)

In [3]:
import panel as pn
import folium
import geopandas
from shapely.geometry import Point
geodictionarypath = pickledir.format("geodict23-12-22.pkl")

def loadpoints(df):
    newdf = df
    if len(df)>300:
        newdf = newdf[:500]
    #pointindex = df_widget.index.to_list
    with open(geodictionarypath, 'rb') as f:
        geodictionary = pickle.load(f)
    searchterms = getsearchterms(newdf)
    coords = searchterms.apply(getcoords, geodict = geodictionary)
    newdf['Coords'] = coords.apply(Point)
    newdf = geopandas.GeoDataFrame(newdf,geometry = "Coords",crs = "EPSG:4326")
    #folium.Map(newdf).add_to(m)
    folium.GeoJson(newdf,na='drop',
                   marker=folium.CircleMarker(
                       radius=4, fill_color="orange", fill_opacity=0.4, color="black", weight=1)
                  ).add_to(m)
    #folium.GeoJson(newdf.to_json(na='drop')).add_to(m)
def getsearchterms(inputdf):
    """returns searchaddress, which can be used with geodict to
    get coords"""
    searchterms = inputdf['Location']+', '+inputdf['City'].replace(EP.citynamedict) +', RI, '
    return searchterms
    
def getcoords(searchaddy, geodict):
    """Takes a search term and returns coordinate points"""
    try:
        return geodict[searchaddy]
    except:
        print(searchaddy)
        return (np.nan,np.nan)    
        
m = folium.Map(location=[41.82, -71.4], zoom_start=11)
loadpoints(df[250000:])
folium_pane = pn.pane.plot.Folium(m, height=400)
folium_pane

/tmp/ipykernel_30348/246073825.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['Coords'] = coords.apply(Point)


Folium(Map, height=400)

In [9]:
grandTotal.droplevel(0)

,PropCount,Assessment,Bedrooms,Hectares
City,,,,
Total,588,14862300,132,3429.883676
Total,1,0,0,2450.776066
Total,99,41945300,557,4.071039
Total,1,556900,2,59.084099
Total,1,138700,0,18.615538
...,...,...,...,...
Total,1,269200,3,4.856227
Total,1,802300,2,0.065032
Total,1,200900,0,0.000000


In [169]:
#Groupby owner and count how many total properties they own
totalprops = replacedowners[['Owner']].assign(City='Total').groupby(['Owner']).value_counts()
totalprops.loc[number_of_cities_owner_is_in>1]
#totalprops.index = totalprops.index.get_level_values('Owner')

In [29]:
df.Owner.unique()

array(['hwang kyujung', 'loveridge charles t', 'hodge steven m', ...,
       'porabik anna', 'khoule oumar', 'winfield ferrucci'], dtype=object)

In [22]:
sumdf.index.get_level_values(level = 0)

Index(['', '', '', '', '', '', '', '', '', '',
       ...
       'zygmunt francis j', 'zylbersztain prevost elsa  and  jackson',
       'zymroz walter  and  joyce', 'zyons mark f', 'zysk david j',
       'zysk tacy l', 'zyskowski carl l', 'zyskowski stanley  and ',
       'zywiak william h', 'zzz llc'],
      dtype='object', length=307669)

In [7]:
sumdf.loc[['zyons mark f','zymroz walter  and  joyce']]

,,PropCount,Assessment,Bedrooms,Hectares
Owner,City,,,,
zyons mark f,Narragansett,1,455100,2,99.957346
zymroz walter and joyce,WestGreen,1,299200,2,0.000000
